In [1]:
import pandas as pd 
import numpy as np 
import extract_vector as ev
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def clean_train_data(train_data, train_label):
    """
    Input: train_data,train_label 
    Ouput: train_data,train_label 
           updated train data and updated train label after removing the nan rows
    """
    n = train_data.shape[0]
    m = train_data.shape[1]
    nan_ll = []
    for i in range(n):
        if(np.isnan(np.sum(train_data[i:i+1, :]))):
            nan_ll.append(i)

    train_data = np.delete(train_data, nan_ll, 0)
    train_label = np.delete(train_label, nan_ll, 0)
    return train_data, train_label

In [3]:
def get_train_labels(train_label):
    for i in train_label:
        if(i[0] > 4.5):
            i[0] = 1
        else:
            i[0] = 0

        if(i[1] > 4.5):
            i[1] = 1
        else:
            i[1] = 0
    return train_label

In [4]:
def get_emotion_label(labels):
    emo = []
    for i in labels:
        if(i[0] == 0 and i[1] == 0):
            emo.append(0)
        elif(i[0] == 1 and i[1] == 0):
            emo.append(1)
        elif(i[0] == 0 and i[1] == 1):
            emo.append(2)
        elif(i[0] == 1 and i[1] == 1):
            emo.append(3)
    return emo

In [5]:
cv = ShuffleSplit(n_splits=5,test_size=0.3,random_state=42)

## Wavelet Entropy Model

In [6]:
train_data = ev.getWaveletEntropyData()
train_label = ev.getLabelData(type='ValAr')

In [7]:
train_data,train_label = clean_train_data(train_data,train_label)
train_label = get_train_labels(train_label)

In [8]:
x_train,x_test,y_train,y_test = train_test_split(train_data,train_label,test_size = 0.3,random_state = 42)

### Arousal

In [9]:
cross_val_score(SVC(),train_data,train_label[:,0],scoring='accuracy',cv=cv)

array([0.72687225, 0.72687225, 0.70044053, 0.69162996, 0.71365639])

In [10]:
arousal_model = SVC()

In [11]:
arousal_model.fit(x_train,y_train[:,0])

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [12]:
pred0 = arousal_model.predict(x_test)

In [13]:
print(classification_report(pred0,y_test[:,0]))

              precision    recall  f1-score   support

         0.0       0.19      0.82      0.31        17
         1.0       0.98      0.72      0.83       210

    accuracy                           0.73       227
   macro avg       0.59      0.77      0.57       227
weighted avg       0.92      0.73      0.79       227



In [14]:
print(accuracy_score(pred0,y_test[:,0]))

0.7268722466960352


### Valence

In [15]:
cross_val_score(SVC(),train_data,train_label[:,1],scoring='accuracy',cv=cv)

array([0.63876652, 0.59030837, 0.63876652, 0.6123348 , 0.6123348 ])

In [16]:
valence_model = SVC()

In [17]:
valence_model.fit(x_train,y_train[:,1])

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [18]:
pred1 = valence_model.predict(x_test)

In [19]:
print(classification_report(pred1,y_test[:,1]))

              precision    recall  f1-score   support

         0.0       0.29      0.48      0.36        48
         1.0       0.83      0.68      0.75       179

    accuracy                           0.64       227
   macro avg       0.56      0.58      0.55       227
weighted avg       0.72      0.64      0.67       227



In [20]:
print(accuracy_score(pred1,y_test[:,1]))

0.6387665198237885


### Emotion

In [21]:
true_emotion = get_emotion_label(y_test)

In [22]:
pred_emotion = np.array(list(zip(pred0,pred1)))
pred_emotion = get_emotion_label(pred_emotion)

In [23]:
confusion_matrix(true_emotion,pred_emotion)

array([[ 0,  4,  3, 13],
       [ 0, 19,  1, 40],
       [ 1,  6, 10, 36],
       [ 1, 17,  1, 75]])

In [24]:
accuracy_score(true_emotion,pred_emotion)

0.4581497797356828

## Fourier Entropy Model

In [25]:
train_data = ev.getFourierEntropyData()
train_label = ev.getLabelData(type='ValAr')

In [26]:
train_data,train_label = clean_train_data(train_data,train_label)
train_label = get_train_labels(train_label)

In [27]:
x_train,x_test,y_train,y_test = train_test_split(train_data,train_label,test_size = 0.2,random_state = 42)

### Arousal

In [28]:
cross_val_score(SVC(),train_data,train_label[:,0],scoring='accuracy',cv=cv)

array([0.6784141 , 0.66079295, 0.63436123, 0.66519824, 0.69162996])

In [29]:
arousal_model = SVC()

In [30]:
arousal_model.fit(x_train,y_train[:,0])

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [31]:
pred0 = arousal_model.predict(x_test)

In [32]:
print(classification_report(pred0,y_test[:,0]))

              precision    recall  f1-score   support

         0.0       0.04      0.50      0.07         4
         1.0       0.98      0.67      0.80       147

    accuracy                           0.67       151
   macro avg       0.51      0.59      0.44       151
weighted avg       0.96      0.67      0.78       151



In [33]:
print(accuracy_score(pred0,y_test[:,0]))

0.6688741721854304


### Valence

In [34]:
cross_val_score(SVC(),train_data,train_label[:,1],scoring='accuracy',cv=cv)

array([0.66519824, 0.60352423, 0.65638767, 0.6123348 , 0.60792952])

In [35]:
valence_model = SVC()

In [36]:
valence_model.fit(x_train,y_train[:,1])

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [37]:
pred1 = valence_model.predict(x_test)

In [38]:
print(classification_report(pred1,y_test[:,1]))

              precision    recall  f1-score   support

         0.0       0.15      0.57      0.24        14
         1.0       0.94      0.66      0.78       137

    accuracy                           0.66       151
   macro avg       0.54      0.62      0.51       151
weighted avg       0.86      0.66      0.73       151



In [39]:
print(accuracy_score(pred1,y_test[:,1]))

0.6556291390728477


### Emotion

In [40]:
true_emotion = get_emotion_label(y_test)

In [41]:
pred_emotion = np.array(list(zip(pred0,pred1)))
pred_emotion = get_emotion_label(pred_emotion)

In [42]:
cm = confusion_matrix(true_emotion,pred_emotion)
cm

array([[ 0,  1,  1, 12],
       [ 0,  7,  1, 32],
       [ 0,  2,  1, 33],
       [ 1,  3,  0, 57]])

In [43]:
accuracy_score(true_emotion,pred_emotion)

0.4304635761589404